# 04 — Merge Mediators (AHEI, FS, PHQ-9)

Run first cell:
```python
%run 00_bootstrap.ipynb
```

In [ ]:

# --- Employment data (OCQ) ---
ocq = pd.read_sas(f"{folder_path}/ocq.sas7bdat")

ocq['employ'] = np.nan
ocq.loc[ocq['OCD150'] == 1, 'employ'] = 1
ocq.loc[(ocq['OCD150'] == 3) | (ocq['OCQ380'] == 5), 'employ'] = 2
ocq.loc[ocq['OCQ380'] == 3, 'employ'] = 3
ocq.loc[ocq['OCQ380'].isin([4, 6]), 'employ'] = 4
ocq.loc[ocq['OCQ380'].isin([1, 2, 7]), 'employ'] = 5
ocq['unemployment'] = (ocq['employ'] == 2).astype(int)
ocq = ocq[['SEQN', 'employ', 'unemployment']]

# --- Housing data (HOQ) ---
hoq = pd.read_sas(f"{folder_path}/hoq.sas7bdat")
hoq.loc[hoq['HOQ065'].isin([7, 9]), 'HOQ065'] = np.nan
hoq = hoq[['SEQN', 'HOD050', 'HOQ065']]

# --- Health insurance (HIQs) ---
hiqs = pd.read_sas(f"{folder_path}/hiqs.sas7bdat")
ins = pd.DataFrame({'SEQN': hiqs['SEQN']})
ins['ins'] = np.nan

# Private insurance
ins.loc[(hiqs['HIQ031A'] == 14) | (hiqs['HID030A'] == 1), 'ins'] = 1

# Medicare
ins.loc[
    ((hiqs['HIQ031B'] == 15) & (hiqs['HIQ031D'] != 17) & (hiqs['HIQ031E'] != 18)) |
    ((hiqs['HID030B'] == 1) & (hiqs['HID030C'] != 1)),
    'ins'
] = 2

# Medicaid
ins.loc[
    (((hiqs['HIQ031D'] == 17) | (hiqs['HIQ031E'] == 18)) & (hiqs['HIQ031B'] != 15)) |
    ((hiqs['HID030B'] != 1) & (hiqs['HID030C'] == 1)),
    'ins'
] = 3
ins.loc[
    ((hiqs['HIQ031B'] == 15) & (hiqs['HIQ031D'] == 17)) |
    ((hiqs['HID030B'] == 1) & (hiqs['HID030C'] == 1)),
    'ins'
] = 3

# Other insurance
ins.loc[
    (hiqs[['HIQ031C', 'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I']].eq(1).any(axis=1)) |
    (hiqs['HID030D'] == 1),
    'ins'
] = 5

# No insurance
ins.loc[(hiqs['HIQ011'] == 2) | (hiqs['HID010'] == 2), 'ins'] = 0

# --- SNAP and Food Security (FSQS) ---
fsqs = pd.read_sas(f"{folder_path}/fsqs.sas7bdat")
snap = pd.DataFrame({'SEQN': fsqs['SEQN'], 'FSDHH': fsqs['FSDHH']})

snap['SNAP'] = np.nan
snap.loc[fsqs['FSQ165'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSQ012'] == 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ012'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSQ171'] == 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ171'] == 2, 'SNAP'] = 0
snap.loc[fsqs['FSD170N'] >= 1, 'SNAP'] = 1
snap.loc[fsqs['FSQ170'] == 1, 'SNAP'] = 1
snap.loc[(fsqs['FSQ170'] == 2) & (fsqs['FSD170N'] < 1), 'SNAP'] = 0
snap.loc[fsqs['FSD200'] == 1, 'SNAP'] = 1

# Food Security
snap['FS'] = np.nan
snap.loc[fsqs['FSDHH'].isin([1, 2]), 'FS'] = 1
snap.loc[fsqs['FSDHH'] > 2, 'FS'] = 0

# Keep only final columns
snap = snap[['SEQN', 'SNAP', 'FSDHH', 'FS']]


<h1>SDOH score</h1>